In [1]:
# import and init of spark session 
from pyspark.sql import SparkSession, DataFrame

# in production we will only include what functions we need 
# however this is R&D so just import everything 
import pyspark.sql.functions as sf

# obviously this is not required in databricks notebooks
spark = SparkSession \
    .builder \
    .appName("EMIS_IA") \
    .getOrCreate()

# based on my threadripper with 24 threads, this can be doubled though
# the default is 200 which is overkill at least for this case
spark.conf.set("spark.sql.shuffle.partitions", 24)

# on 3+ spark, to get out of the box help for skew and spil
# in theory these are enabled by default but in all databricks materials they explicitly enable them#
# so following the norm
spark.conf.set("spark.sql.adaptive.enabled",True)
spark.conf.set("spark.sql.adaptive.skewedJoin.enabled", True)

# for printing data in nice format in jupyter notebooks
# in databricks we will use display() for cool representation, visualizations and many other cool feature like json unfolding, etc. 
# dataframe.show() does not look very nice :)
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

spark

### Some Basic EDA

In [18]:
# let's see our data first
df = spark.read.option("multiline","true").json("data/*")

#display(df.limit(1).toPandas())

#df.show(1)

In [19]:
df.printSchema()

root
 |-- entry: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- fullUrl: string (nullable = true)
 |    |    |-- request: struct (nullable = true)
 |    |    |    |-- method: string (nullable = true)
 |    |    |    |-- url: string (nullable = true)
 |    |    |-- resource: struct (nullable = true)
 |    |    |    |-- abatementDateTime: string (nullable = true)
 |    |    |    |-- activity: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- detail: struct (nullable = true)
 |    |    |    |    |    |    |-- code: struct (nullable = true)
 |    |    |    |    |    |    |    |-- coding: array (nullable = true)
 |    |    |    |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |    |    |    |-- code: string (nullable = true)
 |    |    |    |    |    |    |    |    |    |-- display: string (nullable = true)
 |    |    |    |    |    |   

In [22]:
base_entry_explode = df.withColumn("entries", sf.explode("entry"))
display(base_entry_explode.limit(1).toPandas())

entry  \
0  [(urn:uuid:0d55a582-07fe-a897-776c-3ab5e48cd457, (POST, Patient), (None, None, [Row(city='Chicopee', country='US', extension=[Row(extension=[Row(url='latitude', valueDecimal=42.13458961067817), Row(url='longitude', valueDecimal=-72.59305542210222)], url='http://hl7.org/fhir/StructureDefinition/geolocation')], line=['308 Greenfelder Parade Suite 94'], postalCode='01022', state='MA')], None, None, None, None, None, 1960-07-10, None, None, None, None, None, None, [Row(language=Row(coding=[Row(code='en-US', display='English', system='urn:ietf:bcp:47')], text='English'))], None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, [Row(extension=[Row(url='ombCategory', valueCoding=Row(code='2106-3', display='White', system='urn:oid:2.16.840.1.113883.6.238'), valueString=None), Row(url='text', valueCoding=None, valueString='White')], url='http://hl7.org/fhir/us/core/StructureDefinition/us-core-race', valueAddress=None, valueCode=None, valueDecimal=None, valueString=None), Row(extension=[Row(url='ombCategory', valueCoding=Row(code='2186-5', display='Not Hispanic or Latino', system='urn:oid:2.16.840.1.113883.6.238'), valueString=None), Row(url='text', valueCoding=None, valueString='Not Hispanic or Latino')], url='http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity', valueAddress=None, valueCode=None, valueDecimal=None, valueString=None), Row(extension=None, url='http://hl7.org/fhir/StructureDefinition/patient-mothersMaidenName', valueAddress=None, valueCode=None, valueDecimal=None, valueString='Shaniqua618 Hartmann983'), Row(extension=None, url='http://hl7.org/fhir/us/core/StructureDefinition/us-core-birthsex', valueAddress=None, valueCode='F', valueDecimal=None, valueString=None), Row(extension=None, url='http://hl7.org/fhir/StructureDefinition/patient-birthPlace', valueAddress=Row(city='Webster', country='US', state='Massachusetts'), valueCode=None, valueDecimal=None, valueString=None), Row(extension=None, url='http://synthetichealth.github.io/synthea/disability-adjusted-life-years', valueAddress=None, valueCode=None, valueDecimal=14.837979807308722, valueString=None), Row(extension=None, url='http://synthetichealth.github.io/synthea/quality-adjusted-life-years', valueAddress=None, valueCode=None, valueDecimal=45.16202019269128, valueString=None)], None, female, None, 0d55a582-07fe-a897-776c-3ab5e48cd457, [Row(system='https://github.com/synthetichealth/synthea', type=None, use=None, value='0d55a582-07fe-a897-776c-3ab5e48cd457'), Row(system='http://hospital.smarthealthit.org', type=Row(coding=[Row(code='MR', display='Medical Record Number', system='http://terminology.hl7.org/CodeSystem/v2-0203')], text='Medical Record Number'), use=None, value='0d55a582-07fe-a897-776c-3ab5e48cd457'), Row(system='http://hl7.org/fhir/sid/us-ssn', type=Row(coding=[Row(code='SS', display='Social Security Number', system='http://terminology.hl7.org/CodeSystem/v2-0203')], text='Social Security Number'), use=None, value='999-20-9087'), Row(system='urn:oid:2.16.840.1.113883.4.3.25', type=Row(coding=[Row(code='DL', display="Driver's License", system='http://terminology.hl7.org/CodeSystem/v2-0203')], text="Driver's License"), use=None, value='S99957903'), Row(system='http://standardhealthrecord.org/fhir/StructureDefinition/passportNumber', type=Row(coding=[Row(code='PPN', display='Passport Number', system='http://terminology.hl7.org/CodeSystem/v2-0203')], text='Passport Number'), use=None, value='X4871961X')], None, None, None, None, None, None, None, None, None, Row(coding=[Row(code='S', display='S', system='http://terminology.hl7.org/CodeSystem/v3-MaritalStatus')], text='S'), None, None, Row(profile=['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient']), False, None, [Row(family='Hahn503', given=['Gilma310'], prefix=['Ms.'], suffix=None, use='official')], None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

### Conclusion

* This is a very very bad test
* You provide a very complex production schema which requires business discussion with data scientiests how to explode the data
* I am a bit lost what you are trying to test here ? 
* There is no domain knowledge; so are we suppose to guess at what level to go deep ?
* This is not 1 hour test
* Some advices on what we test in interviews:
* Ability to be able to have basic coding skills
* Ability to use the proper state of the art techniques
* Ability to write some comments and show that code is organised

### Many thanks